In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,62,2024-03-04,-5,106.065574,97.770492,0.514459,12.722951,21.900000,0.183590,107.981967,...,10.701667,24.776667,0.174333,115.206667,0,3,0,2,15,2024
1,59,2024-03-04,1,117.844828,97.167241,0.568155,11.213793,24.779310,0.216638,121.063793,...,10.929508,21.886885,0.221180,120.532787,0,3,0,3,13,2024
2,60,2024-03-04,2,107.491525,96.705085,0.503966,12.525424,26.788136,0.194864,109.910169,...,12.763333,22.953333,0.220933,116.811667,0,3,0,2,25,2024
3,61,2024-03-04,-14,114.616667,102.721667,0.538900,11.780000,19.938333,0.171500,111.370000,...,13.034426,28.772131,0.213918,117.347541,0,3,0,3,30,2024
4,61,2024-03-04,-1,111.650000,95.443333,0.529967,10.588333,24.661667,0.188283,115.048333,...,11.208475,23.567797,0.172356,118.089831,0,3,0,3,4,2024
5,60,2024-03-04,-1,121.237288,99.137288,0.578712,10.511864,20.588136,0.205542,121.461017,...,11.716129,19.470968,0.210306,115.966129,0,3,0,4,21,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-04,MEM,BRK,4.300000,1.235294,0
1,2024-03-04,LAC,MIL,2.640000,1.505051,0
2,2024-03-04,POR,MIN,8.500000,1.083333,0
3,2024-03-04,WAS,UTA,2.850000,1.444444,0
4,2024-03-04,CHI,SAC,2.950000,1.420168,0
5,2024-03-04,OKC,LAL,1.980392,1.847458,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-04,MEM,BRK,4.300000,1.235294,0
1,2024-03-04,LAC,MIL,2.640000,1.505051,0
2,2024-03-04,POR,MIN,8.500000,1.083333,0
3,2024-03-04,WAS,UTA,2.850000,1.444444,0
4,2024-03-04,CHI,SAC,2.950000,1.420168,0
5,2024-03-04,OKC,LAL,1.980392,1.847458,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-04,MEM,BRK,4.300000,1.235294,0
1,2024-03-04,LAC,MIL,2.640000,1.505051,0
2,2024-03-04,POR,MIN,8.500000,1.083333,0
3,2024-03-04,WAS,UTA,2.850000,1.444444,0
4,2024-03-04,CHI,SAC,2.950000,1.420168,0
5,2024-03-04,OKC,LAL,1.980392,1.847458,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-04,MEM,BRK,4.300000,1.235294,0
1,2024-03-04,LAC,MIL,2.640000,1.505051,0
2,2024-03-04,POR,MIN,8.500000,1.083333,0
3,2024-03-04,WAS,UTA,2.850000,1.444444,0
4,2024-03-04,CHI,SAC,2.950000,1.420168,0
5,2024-03-04,OKC,LAL,1.980392,1.847458,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-04,MEM,BRK,4.300000,1.235294,0.0
1,2024-03-04,LAC,MIL,2.640000,1.505051,1.0
2,2024-03-04,POR,MIN,8.500000,1.083333,0.0
3,2024-03-04,WAS,UTA,2.850000,1.444444,0.0
4,2024-03-04,CHI,SAC,2.950000,1.420168,0.0
5,2024-03-04,OKC,LAL,1.980392,1.847458,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-04,MEM,BRK,4.300000,1.235294,0.0
1,2024-03-04,LAC,MIL,2.640000,1.505051,1.0
2,2024-03-04,POR,MIN,8.500000,1.083333,0.0
3,2024-03-04,WAS,UTA,2.850000,1.444444,0.0
4,2024-03-04,CHI,SAC,2.950000,1.420168,0.0
5,2024-03-04,OKC,LAL,1.980392,1.847458,1.0
